In [1]:
# basic dependencies

import numpy as np
from numpy import loadtxt
from numpy import savetxt

import pandas as pd
import math
import time

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

###########

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly

torch.set_printoptions(precision=3)

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP, FixedNoiseGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.optim.optimize import optimize_acqf
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement

# utilities
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.utils.multi_objective.hypervolume import infer_reference_point
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

###########

# pymoo dependencies
import pymoo

from pymoo.factory import get_problem
from pymoo.core.problem import ElementwiseProblem

from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.unsga3 import UNSGA3
from pymoo.algorithms.moo.ctaea import CTAEA
from pymoo.factory import get_sampling, get_crossover, get_mutation, get_reference_directions, get_termination
from pymoo.optimize import minimize

from pymoo.core.problem import Problem
from pymoo.util.termination.no_termination import NoTermination
from pymoo.core.evaluator import set_cv
from pymoo.factory import get_performance_indicator


###########

# plotting dependencies
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

# this is for the colorbar, you can change the cmap if you prefer other colour schemes
from matplotlib.cm import ScalarMappable
cm = plt.cm.get_cmap('viridis')

# function to return the std dev across runs
def ci(y, N_TRIALS):
    return 1.96 * y.std(axis=0) / np.sqrt(N_TRIALS)

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, WhiteKernel
from sklearn.preprocessing import MinMaxScaler

In [2]:
def optimize_qnehvi(problem, ref_point, initial_x, # must haves
                    N_BATCH, BATCH_SIZE, 
                    random_state=torch.randint(1000000, (1,)).item(), noise=0, verbose=False): # change noise here!
    
    print("Optimizing with qNEHVI")

    t0 = time.time()

    # some initializing 
    torch.manual_seed(random_state) # gives a consistent seed based on the trial number
    hv=Hypervolume(ref_point=-ref_point) # sets the hv based on problem, flip since BoTorch takes maximisation
    hvs = [] # create a blank array to append the scores at each batch/iteration for that run
    
    ##########
    # generate initial training data for that run
    train_x = initial_x
    train_obj, train_con = problem.evaluate(train_x)

    # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
    train_obj_noisy = train_obj + noise*torch.randn_like(train_obj)
    train_con_noisy = train_con + noise*torch.randn_like(train_con)
    
    ##########
    
    # normalize inputs to [0,1] first before feeding into model
    standard_bounds = torch.zeros(2, problem.n_var, **tkwargs)
    standard_bounds[1] = 1
    train_x_gp = normalize(train_x, standard_bounds)
    
    # form the output train_y data by concentenating ground truth train_obj with its feasibility value on the rightmost
    # this is necessary since the surrogate GpyTorch model needs to model BOTH obj and con for predicted candidates
    train_y = torch.cat([train_obj_noisy, train_con_noisy], dim=-1) # model takes noisy observations

    # define and train surrogate models for objective and constraint
    models = []
    for i in range(train_y.shape[-1]):
        models.append(SingleTaskGP(train_x_gp, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
        
    ##########    
    
    def create_idxr(i):
        def idxr(Z):
            return Z[..., i]

        return idxr

    def create_idxrs():
        return [create_idxr(i=i) for i in range(problem.n_obj, problem.n_obj+problem.n_constr)]
    
    # original location for an extra HV check wrt to initial samples
    
    ########## ########## ########## start of iteration loop


    # training loop for N_BATCH iterations
    for iteration in range(1, N_BATCH + 1):    

        t3 = time.time()
                
        # fit the surrogate model
        fit_gpytorch_model(mll)    
                
        ##########
            
        # define the acqusition function for EIC if feas_weighting is false
        acq_func = qNoisyExpectedHypervolumeImprovement(
            model=model,
            ref_point=-ref_point, # for computing HV, must flip for BoTorch
            X_baseline=train_x, # feed total list of train_x for this current iteration
            sampler=SobolQMCNormalSampler(num_samples=128),  # determines how candidates are randomly proposed before selection
            objective=IdentityMCMultiOutputObjective(outcomes=np.arange(problem.n_obj).tolist()), # optimize first n_obj col 
            constraints=create_idxrs(), # constraint on last n_constr col
            prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on
        
        ##########
        
        # propose candidates given defined qNEHVI acq func given model and latest observed training data
        new_x, _ = optimize_acqf(
                        acq_function=acq_func,
                        bounds=standard_bounds, # since train_x was normalized
                        q=BATCH_SIZE, # no of candidates to propose in parallel
                        num_restarts=2, # no of restarts if q candidates fail to show improvement
                        raw_samples=128,  # pool of samples to choose the starting points from
                        options={"batch_limit": 5, "maxiter": 200}, # default arguments, not too sure about this yet
                        )

        # unormalize our training inputs back to original problem bounds
        new_x =  unnormalize(new_x.detach(), bounds=problem.bounds)

        # feed new proposed observations into objective func to get its new ground truth
        new_obj, new_con = problem.evaluate(new_x)

        # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
        new_obj_noisy = new_obj + noise*torch.randn_like(new_obj)
        new_con_noisy = new_con + noise*torch.randn_like(new_con)

        # update training points by concatenating the new values into their respective tensors
        train_x = torch.cat([train_x, new_x])
        train_obj = torch.cat([train_obj, new_obj])
        train_con = torch.cat([train_con, new_con])
        train_obj_noisy = torch.cat([train_obj_noisy, new_obj_noisy])
        train_con_noisy = torch.cat([train_con_noisy, new_con_noisy])
        
        ##########
        
        # computing HV of current candidate list
        is_feas = (train_con <= 0).all(dim=-1) # check whether points fit ALL (.all) constraint criteria
        feas_train_obj = train_obj[is_feas] # take only points that fit the 1st check
        if feas_train_obj.shape[0] > 0:
            pareto_mask = is_non_dominated(feas_train_obj) # check for 2nd criteria: non-dominated, meaning new pareto optimal
            pareto_y = feas_train_obj[pareto_mask] # take only points that fit the 2nd check
            volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point
        else:
            volume = 0.0
        
        hvs.append(volume)
        
        ##########

        # update the surrogate models for next iteration
        train_x_gp = normalize(train_x, standard_bounds) # dont forget to renormalize!
        train_y = torch.cat([train_obj_noisy, train_con_noisy], dim=-1) # model takes noisy observations

        models = []
        for i in range(train_y.shape[-1]):
            models.append(SingleTaskGP(train_x, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
        model = ModelListGP(*models)
        mll = SumMarginalLogLikelihood(model.likelihood, model)
        
        ##########
        
        t4 = time.time()
        if verbose:
            print(
                    f"Batch {iteration:>2} of {N_BATCH}: Hypervolume = "
                    f"{hvs[-1]:>4.2f}, "
                    f"time = {t4-t3:>4.2f}s.\n"
                    , end="")
            
        del new_x, new_obj, new_con, new_obj_noisy, new_con_noisy, train_x_gp, train_y
        torch.cuda.empty_cache() # clear some memory here between each run/trial     
        
        ########## ########## ########## end of iteration loop

    t1 = time.time()
    print(f"Time taken in total: {t1-t0:>4.2f}s.")       
    
    # returns the HV score across iterations, total training set as an array
    return hvs, torch.hstack([train_x, train_obj, train_con]).cpu().numpy()

def optimize_nsga3(problem, ref_point, initial_x, # must haves
                   N_BATCH, pop_size,
                   ref_num=10, # as a rule of thumb, pop_size>ref_num,
                   noise=0, random_state=np.random.randint(0, 1000000, (1,)).item(), verbose=False):
    
    print("Optimizing with NSGA-III")
    
    t0 = time.time()    
    
    # some initializing
    hv=Hypervolume(ref_point=-ref_point) # sets the hv based on problem, flip since BoTorch takes maximisation
    hvs = [] # create a blank array to append the scores at each batch/iteration for that run
    
    ##########
    
    noise=noise

    # define a pymoo problem class based on the torch class
    class PymooProblem(ElementwiseProblem):

        def __init__(self):
            super().__init__(n_var=problem.n_var,
                             n_obj=problem.n_obj,
                             n_constr=problem.n_constr,
                             xl=problem.bounds[0].cpu().numpy(),
                             xu=problem.bounds[1].cpu().numpy())

        def _evaluate(self, X, out, *args, **kwargs):
            # base input/output from torch class
            train_x = torch.tensor(X.tolist(), **tkwargs).unsqueeze(dim=0)
            
            # take the noisy observations for algo
            train_obj, train_con = problem.evaluate(train_x)
            train_obj_noisy = train_obj + noise*torch.randn_like(train_obj)
            train_con_noisy = train_con + noise*torch.randn_like(train_con)

            # output the noisy observations instead
            out["F"] = -train_obj_noisy.cpu().numpy() # flip since botorch assumes maximisation vs pymoo minimization
            out["G"] = train_con_noisy.cpu().numpy()
            
    ##########        

    pymooproblem = PymooProblem()

    # create the reference directions to be used for the optimization
    ref_dirs = get_reference_directions("energy", problem.n_obj, ref_num, seed=random_state)
    
    # initial sampling
    sampling = initial_x.cpu().numpy()
    
    # create the algorithm object
    algorithm = UNSGA3(pop_size=pop_size,
                      ref_dirs=ref_dirs,
                      sampling=sampling
                     )

    # execute the optimization, take N_BATCH+1 since 1st iteration is just the initial sampling
    res = minimize(pymooproblem, algorithm, seed=random_state, termination=('n_gen', N_BATCH+1),
                   verbose=verbose, save_history=True)
    
    ##########
    
    t1 = time.time()
    print(f"Time taken in total: {t1-t0:>4.2f}s.")

    ##########
        
    # convert population data into tensor form
    # initial data
    train_x = torch.tensor(res.history[0].pop.get("X").tolist(), **tkwargs)
    train_obj, train_con = problem.evaluate(train_x) 
    # population at each iteration
    for i in range(1,N_BATCH+1): # don't forget we did +1 for total iterations
        new_x = torch.tensor(res.history[i].pop.get("X").tolist(), **tkwargs)
        new_obj, new_con = problem.evaluate(new_x)
        train_x = torch.cat([train_x, new_x])
        train_obj = torch.cat([train_obj, new_obj])
        train_con = torch.cat([train_con, new_con])

    ##########
        
    # calculate hypervolume based on checks, done on noiseless train_obj and train_con
    iterdict = {}
    a = 0
    b = initial_x.shape[0]
    for i in range(0, N_BATCH+1):

        iterdict[i] = (a, b)
        # a stays at zero
        b+=pop_size
    
    for i in range(0,N_BATCH):
        hvs_obj = train_obj[iterdict[i][0]:iterdict[i][1]]
        hvs_con = train_con[iterdict[i][0]:iterdict[i][1]]

        is_feas = (hvs_con <= 0).all(dim=-1) # check whether points fit ALL (.all) constraint criteria
        feas_train_obj = hvs_obj[is_feas] # take only points that fit the 1st check
        if feas_train_obj.shape[0] > 0:
            pareto_mask = is_non_dominated(feas_train_obj) # check for 2nd criteria: non-dominated, meaning new pareto optimal
            pareto_y = feas_train_obj[pareto_mask] # take only points that fit the 2nd check
            volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point
        else:
            volume = 0.0
        
        hvs.append(volume)
        
    print("DONE!\n")
        
    return hvs, torch.hstack([train_x, train_obj, train_con]).cpu().numpy()

def optimize_qnehvi_noconstr(problem, ref_point, initial_x, # must haves
                    N_BATCH, BATCH_SIZE, 
                    random_state=torch.randint(1000000, (1,)).item(), noise=0, verbose=False): # change noise here!
    
    print("Optimizing with qNEHVI")

    t0 = time.time()

    # some initializing 
    torch.manual_seed(random_state) # gives a consistent seed based on the trial number
    hv=Hypervolume(ref_point=-ref_point) # sets the hv based on problem, flip since BoTorch takes maximisation
    hvs = [] # create a blank array to append the scores at each batch/iteration for that run
    
    ##########
    # generate initial training data for that run
    train_x = initial_x
    train_obj = problem.evaluate(train_x)

    # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
    train_obj_noisy = train_obj + noise*torch.randn_like(train_obj)
    
    ##########
    
    # normalize inputs to [0,1] first before feeding into model
    standard_bounds = torch.zeros(2, problem.n_var, **tkwargs)
    standard_bounds[1] = 1
    train_x_gp = normalize(train_x, standard_bounds)
    
    # form the output train_y data by concentenating ground truth train_obj with its feasibility value on the rightmost
    # this is necessary since the surrogate GpyTorch model needs to model BOTH obj and con for predicted candidates
    train_y = train_obj_noisy

    # define and train surrogate models for objective and constraint
    models = []
    for i in range(train_y.shape[-1]):
        models.append(SingleTaskGP(train_x_gp, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
        
    ##########    
    
    # original location for an extra HV check wrt to initial samples
    
    ########## ########## ########## start of iteration loop


    # training loop for N_BATCH iterations
    for iteration in range(1, N_BATCH + 1):    

        t3 = time.time()
                
        # fit the surrogate model
        fit_gpytorch_model(mll)    
                
        ##########
            
        # define the acqusition function for EIC if feas_weighting is false
        acq_func = qNoisyExpectedHypervolumeImprovement(
            model=model,
            ref_point=-ref_point, # for computing HV, must flip for BoTorch
            X_baseline=train_x, # feed total list of train_x for this current iteration
            sampler=SobolQMCNormalSampler(num_samples=128),  # determines how candidates are randomly proposed before selection
            objective=IdentityMCMultiOutputObjective(outcomes=np.arange(problem.n_obj).tolist()), # optimize first n_obj col 
            prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on
        
        ##########
        
        # propose candidates given defined qNEHVI acq func given model and latest observed training data
        new_x, _ = optimize_acqf(
                        acq_function=acq_func,
                        bounds=standard_bounds, # since train_x was normalized
                        q=BATCH_SIZE, # no of candidates to propose in parallel
                        num_restarts=2, # no of restarts if q candidates fail to show improvement
                        raw_samples=128,  # pool of samples to choose the starting points from
                        options={"batch_limit": 5, "maxiter": 200}, # default arguments, not too sure about this yet
                        )

        # unormalize our training inputs back to original problem bounds
        new_x =  unnormalize(new_x.detach(), bounds=problem.bounds)

        # feed new proposed observations into objective func to get its new ground truth
        new_obj = problem.evaluate(new_x)

        # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
        new_obj_noisy = new_obj + noise*torch.randn_like(new_obj)

        # update training points by concatenating the new values into their respective tensors
        train_x = torch.cat([train_x, new_x])
        train_obj = torch.cat([train_obj, new_obj])
        train_obj_noisy = torch.cat([train_obj_noisy, new_obj_noisy])
        
        ##########
        
        # computing HV of current candidate list
        pareto_mask = is_non_dominated(train_obj) # check for 2nd criteria: non-dominated, meaning new pareto optimal
        pareto_y = train_obj[pareto_mask] # take only points that fit the 2nd check
        volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point
        
        hvs.append(volume)
        
        ##########

        # update the surrogate models for next iteration
        train_x_gp = normalize(train_x, standard_bounds) # dont forget to renormalize!
        train_y = train_obj_noisy

        models = []
        for i in range(train_y.shape[-1]):
            models.append(SingleTaskGP(train_x, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
        model = ModelListGP(*models)
        mll = SumMarginalLogLikelihood(model.likelihood, model)
        
        ##########
        
        t4 = time.time()
        if verbose:
            print(
                    f"Batch {iteration:>2} of {N_BATCH}: Hypervolume = "
                    f"{hvs[-1]:>4.2f}, "
                    f"time = {t4-t3:>4.2f}s.\n"
                    , end="")
            
        del new_x, new_obj, new_obj_noisy, train_x_gp, train_y
        torch.cuda.empty_cache() # clear some memory here between each run/trial     
        
        ########## ########## ########## end of iteration loop

    t1 = time.time()
    print(f"Time taken in total: {t1-t0:>4.2f}s.")       
    
    # returns the HV score across iterations, total training set as an array
    return hvs, torch.hstack([train_x, train_obj]).cpu().numpy()

def optimize_nsga3_noconstr(problem, ref_point,  initial_x, # must haves
                            N_BATCH, pop_size,
                            ref_num=10, # as a rule of thumb, pop_size>ref_num,
                            noise=0, random_state=np.random.randint(0, 1000000, (1,)).item(), verbose=False):
    
    print("Optimizing with NSGA-III")
    
    t0 = time.time()    
    
    # some initializing
    hv=Hypervolume(ref_point=-ref_point) # sets the hv based on problem, flip since BoTorch takes maximisation
    hvs = [] # create a blank array to append the scores at each batch/iteration for that run
    
    ##########
    
    noise=noise

    # define a pymoo problem class based on the torch class
    class PymooProblem(ElementwiseProblem):

        def __init__(self):
            super().__init__(n_var=problem.n_var,
                             n_obj=problem.n_obj,
                             xl=problem.bounds[0].cpu().numpy(),
                             xu=problem.bounds[1].cpu().numpy())

        def _evaluate(self, X, out, *args, **kwargs):
            # base input/output from torch class
            train_x = torch.tensor(X.tolist(), **tkwargs).unsqueeze(dim=0)
            
            # take the noisy observations for algo
            train_obj = problem.evaluate(train_x)
            train_obj_noisy = train_obj + noise*torch.randn_like(train_obj)

            # output the noisy observations instead
            out["F"] = -train_obj_noisy.cpu().numpy() # flip since botorch assumes maximisation vs pymoo minimization
            
    ##########        

    pymooproblem = PymooProblem()

    # create the reference directions to be used for the optimization
    ref_dirs = get_reference_directions("energy", problem.n_obj, ref_num, seed=random_state)
    
    # initial sampling
    sampling = initial_x.cpu().numpy()
    
    # create the algorithm object
    algorithm = UNSGA3(pop_size=pop_size,
                      ref_dirs=ref_dirs,
                      sampling=sampling
                     )

    # execute the optimization, take N_BATCH+1 since 1st iteration is just the initial sampling
    res = minimize(pymooproblem, algorithm, seed=random_state, termination=('n_gen', N_BATCH+1),
                   verbose=verbose, save_history=True)
    
    ##########
    
    t1 = time.time()
    print(f"Time taken in total: {t1-t0:>4.2f}s.")
    
    ##########
    
    # convert population data into tensor form
    # initial data
    train_x = torch.tensor(res.history[0].pop.get("X").tolist(), **tkwargs)
    train_obj = problem.evaluate(train_x) 
    # population at each iteration
    for i in range(1,N_BATCH+1): # don't forget we did +1 for total iterations
        new_x = torch.tensor(res.history[i].pop.get("X").tolist(), **tkwargs)
        new_obj = problem.evaluate(new_x)
        train_x = torch.cat([train_x, new_x])
        train_obj = torch.cat([train_obj, new_obj])
        
    ##########
    
    # calculate hypervolume based on checks, done on noiseless train_obj and train_con
    iterdict = {}
    a = 0
    b = initial_x.shape[0]
    for i in range(0, N_BATCH+1):

        iterdict[i] = (a, b)
        # a stays at zero
        b+=pop_size
    
    for i in range(0,N_BATCH):
        hvs_obj = train_obj[iterdict[i][0]:iterdict[i][1]]

        # computing HV of current candidate list
        pareto_mask = is_non_dominated(hvs_obj) # check for 2nd criteria: non-dominated, meaning new pareto optimal
        pareto_y = hvs_obj[pareto_mask] # take only points that fit the 2nd check
        volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point
                
        hvs.append(volume)   
        
    ##########
     
    return hvs, torch.hstack([train_x, train_obj]).cpu().numpy()

In [3]:
dimensions = 8

from botorch.test_functions.multi_objective import ZDT1

ZDT1base = ZDT1(dim=dimensions, negate=True).to(**tkwargs)

class Problem_ZDT1(torch.nn.Module):
    n_var = dimensions
    n_obj = 2
    n_constr = 1 # inequality constraints only!
        
    bounds = torch.vstack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])
    
    def evaluate(X):        
    
        output = ZDT1base(X)
        
        # for 1 constraint, take c1.unsqueeze(dim=-1)
        # for >1 constraint, take torch.stack([c1, c2....], dim=-1)
        
        return output
    
from botorch.test_functions.multi_objective import ZDT2

ZDT2base = ZDT2(dim=dimensions, negate=True).to(**tkwargs)

class Problem_ZDT2(torch.nn.Module):
    n_var = dimensions
    n_obj = 2
    n_constr = 1 # inequality constraints only!
        
    bounds = torch.vstack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])
    
    def evaluate(X):        
    
        output = ZDT2base(X)
        
        # for 1 constraint, take c1.unsqueeze(dim=-1)
        # for >1 constraint, take torch.stack([c1, c2....], dim=-1)
        
        return output
    
from botorch.test_functions.multi_objective import ZDT3

ZDT3base = ZDT3(dim=dimensions, negate=True).to(**tkwargs)

class Problem_ZDT3(torch.nn.Module):
    n_var = dimensions
    n_obj = 2
    n_constr = 1 # inequality constraints only!
        
    bounds = torch.vstack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])
    
    def evaluate(X):        
    
        output = ZDT3base(X)
        
        # for 1 constraint, take c1.unsqueeze(dim=-1)
        # for >1 constraint, take torch.stack([c1, c2....], dim=-1)
        
        return output
    
from botorch.test_functions.multi_objective import MW7

MW7base = MW7(dim=dimensions, negate=True).to(**tkwargs)

class Problem_MW7(torch.nn.Module):
    n_var = dimensions
    n_obj = 2
    n_constr = 1
    
    bounds = MW7base.bounds    
    
    def evaluate(X):       
        output = MW7base(X)
        slack = -MW7base.evaluate_slack(X)
       
        return output, slack

In [4]:
ref_point = torch.tensor([11,11], **tkwargs)

N_TRIALS = 10
verbose = True
noise = 0.00

hvs_qnehvi_all0 = []
hvs_qnehvi_all1 = []
hvs_qnehvi_all2 = []

hvs_nsga3_all0 = []
hvs_nsga3_all1 = []
hvs_nsga3_all2 = []
hvs_nsga3_all3 = []

train_qnehvi_all = []
train_nsga3_all = []

problem = Problem_ZDT1

# main loop for each trial/run, random_state will be trial number
for trial in range(1, 10 + 1):
    print(f"\nTrial {trial:>2} of {N_TRIALS} for problem {problem} with d = {dimensions}\n", end="")

    # initialize with a 2*(d+1) sample set

    initial_x = draw_sobol_samples(bounds=problem.bounds,n=1, q=2*(problem.n_var+1), seed=trial).squeeze(0)

    ###############    
    
    hvs_qnehvi, train_qnehvi = optimize_qnehvi_noconstr(problem, ref_point, initial_x,
                                                        N_BATCH=96, BATCH_SIZE=2,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all0.append(hvs_qnehvi)
        
    ###############    

    hvs_qnehvi, train_qnehvi = optimize_qnehvi_noconstr(problem, ref_point, initial_x,
                                                        N_BATCH=48, BATCH_SIZE=4,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all1.append(hvs_qnehvi)

    ###############    

    hvs_qnehvi, train_qnehvi = optimize_qnehvi_noconstr(problem, ref_point, initial_x,
                                                        N_BATCH=24, BATCH_SIZE=8,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all2.append(hvs_qnehvi)
    train_qnehvi_all.append(train_qnehvi)
    
    ###############    

    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=96, pop_size=2, ref_num=2,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all0.append(hvs_nsga3) 
    
    ###############

    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=48, pop_size=4, ref_num=4,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all1.append(hvs_nsga3) 
    
    ###############
    
    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=24, pop_size=8, ref_num=8,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all2.append(hvs_nsga3) 
    train_nsga3_all.append(train_nsga3)

    ###############
    
    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=12, pop_size=16, ref_num=16,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all3.append(hvs_nsga3) 
    
    ###############

savetxt("ZDT1_hvs_qnehvi_96by2.csv", hvs_qnehvi_all0, delimiter=',')
savetxt("ZDT1_hvs_qnehvi_48by4.csv", hvs_qnehvi_all1, delimiter=',')
savetxt("ZDT1_hvs_qnehvi_24by8.csv", hvs_qnehvi_all2, delimiter=',')    

savetxt("ZDT1_hvs_nsga3_96by2.csv", hvs_nsga3_all0, delimiter=',')
savetxt("ZDT1_hvs_nsga3_48by4.csv", hvs_nsga3_all1, delimiter=',')
savetxt("ZDT1_hvs_nsga3_24by8.csv", hvs_nsga3_all2, delimiter=',')
savetxt("ZDT1_hvs_nsga3_12by16.csv", hvs_nsga3_all3, delimiter=',')

savetxt("ZDT1_train_qnehvi_24by8.csv", np.array(train_qnehvi_all).reshape(-1), delimiter=',')
savetxt("ZDT1_train_nsga3_24by8.csv", np.array(train_nsga3_all).reshape(-1), delimiter=',')
    
print("ALL DONE!")


Trial  1 of 10 for problem <class '__main__.Problem_ZDT1'> with d = 8
Optimizing with qNEHVI


C:\Users\user\anaconda3\lib\site-packages\botorch\fit.py:143: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is C:\Users\user\AppData\Local\Temp/torch/kernels. This warning will appear only once per process. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\jit_utils.cpp:860.)
  warnings.warn(w.message, w.category)
C:\Users\user\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:1672.)
  Linv = torch.triangular_solve(Eye, 

Batch  1 of 96: Hypervolume = 91.65, time = 4.06s.
Batch  2 of 96: Hypervolume = 93.47, time = 2.92s.
Batch  3 of 96: Hypervolume = 96.74, time = 2.49s.
Batch  4 of 96: Hypervolume = 100.75, time = 2.96s.
Batch  5 of 96: Hypervolume = 105.68, time = 2.28s.
Batch  6 of 96: Hypervolume = 109.28, time = 2.53s.
Batch  7 of 96: Hypervolume = 112.30, time = 3.96s.
Batch  8 of 96: Hypervolume = 115.22, time = 2.82s.
Batch  9 of 96: Hypervolume = 116.20, time = 2.10s.
Batch 10 of 96: Hypervolume = 116.66, time = 1.68s.
Batch 11 of 96: Hypervolume = 116.68, time = 1.90s.
Batch 12 of 96: Hypervolume = 116.69, time = 2.79s.
Batch 13 of 96: Hypervolume = 116.69, time = 3.28s.
Batch 14 of 96: Hypervolume = 116.69, time = 3.97s.
Batch 15 of 96: Hypervolume = 116.73, time = 3.28s.
Batch 16 of 96: Hypervolume = 116.73, time = 4.43s.
Batch 17 of 96: Hypervolume = 118.70, time = 5.30s.
Batch 18 of 96: Hypervolume = 118.90, time = 2.90s.
Batch 19 of 96: Hypervolume = 118.95, time = 1.97s.
Batch 20 of 96:

Batch 13 of 24: Hypervolume = 120.49, time = 8.92s.
Batch 14 of 24: Hypervolume = 120.49, time = 8.90s.
Batch 15 of 24: Hypervolume = 120.49, time = 9.50s.
Batch 16 of 24: Hypervolume = 120.49, time = 9.48s.
Batch 17 of 24: Hypervolume = 120.49, time = 8.24s.
Batch 18 of 24: Hypervolume = 120.49, time = 8.68s.
Batch 19 of 24: Hypervolume = 120.49, time = 8.56s.
Batch 20 of 24: Hypervolume = 120.49, time = 5.87s.
Batch 21 of 24: Hypervolume = 120.49, time = 9.62s.
Batch 22 of 24: Hypervolume = 120.49, time = 7.84s.
Batch 23 of 24: Hypervolume = 120.49, time = 10.66s.
Batch 24 of 24: Hypervolume = 120.49, time = 6.52s.
Time taken in total: 187.19s.
Optimizing with NSGA-III
Time taken in total: 0.78s.
Optimizing with NSGA-III
Time taken in total: 0.72s.
Optimizing with NSGA-III
Time taken in total: 0.77s.
Optimizing with NSGA-III
Time taken in total: 0.76s.

Trial  2 of 10 for problem <class '__main__.Problem_ZDT1'> with d = 8
Optimizing with qNEHVI
Batch  1 of 96: Hypervolume = 97.78, ti

Batch 44 of 48: Hypervolume = 119.87, time = 4.97s.
Batch 45 of 48: Hypervolume = 119.87, time = 2.46s.
Batch 46 of 48: Hypervolume = 119.87, time = 4.93s.
Batch 47 of 48: Hypervolume = 119.87, time = 6.72s.
Batch 48 of 48: Hypervolume = 119.87, time = 2.91s.
Time taken in total: 191.97s.
Optimizing with qNEHVI
Batch  1 of 24: Hypervolume = 99.75, time = 5.80s.
Batch  2 of 24: Hypervolume = 106.99, time = 4.81s.
Batch  3 of 24: Hypervolume = 113.82, time = 6.60s.
Batch  4 of 24: Hypervolume = 118.47, time = 6.46s.
Batch  5 of 24: Hypervolume = 119.06, time = 6.69s.
Batch  6 of 24: Hypervolume = 119.07, time = 6.79s.
Batch  7 of 24: Hypervolume = 119.07, time = 7.33s.
Batch  8 of 24: Hypervolume = 119.07, time = 6.47s.
Batch  9 of 24: Hypervolume = 119.07, time = 6.71s.
Batch 10 of 24: Hypervolume = 119.07, time = 3.90s.
Batch 11 of 24: Hypervolume = 119.07, time = 8.62s.
Batch 12 of 24: Hypervolume = 119.07, time = 7.91s.
Batch 13 of 24: Hypervolume = 119.07, time = 8.95s.
Batch 14 of 

...

Batch 51 of 96: Hypervolume = 119.74, time = 3.91s.
Batch 52 of 96: Hypervolume = 119.74, time = 3.96s.
Batch 53 of 96: Hypervolume = 119.74, time = 2.74s.
Batch 54 of 96: Hypervolume = 119.74, time = 2.35s.
Batch 55 of 96: Hypervolume = 119.74, time = 2.84s.
Batch 56 of 96: Hypervolume = 119.74, time = 2.81s.
Batch 57 of 96: Hypervolume = 119.74, time = 4.23s.
Batch 58 of 96: Hypervolume = 119.74, time = 4.91s.
Batch 59 of 96: Hypervolume = 119.74, time = 2.62s.
Batch 60 of 96: Hypervolume = 119.74, time = 2.87s.
Batch 61 of 96: Hypervolume = 119.74, time = 2.51s.
Batch 62 of 96: Hypervolume = 119.74, time = 1.92s.
Batch 63 of 96: Hypervolume = 119.74, time = 2.94s.
Batch 64 of 96: Hypervolume = 119.74, time = 4.28s.
Batch 65 of 96: Hypervolume = 119.74, time = 3.42s.
Batch 66 of 96: Hypervolume = 119.74, time = 2.67s.
Batch 67 of 96: Hypervolume = 119.74, time = 2.90s.
Batch 68 of 96: Hypervolume = 119.74, time = 2.80s.
Batch 69 of 96: Hypervolume = 119.74, time = 3.11s.
Batch 70 of 

Batch 33 of 96: Hypervolume = 119.02, time = 2.87s.
Batch 34 of 96: Hypervolume = 119.02, time = 2.28s.
Batch 35 of 96: Hypervolume = 119.02, time = 2.56s.
Batch 36 of 96: Hypervolume = 119.02, time = 1.99s.
Batch 37 of 96: Hypervolume = 119.02, time = 4.23s.
Batch 38 of 96: Hypervolume = 119.02, time = 4.15s.
Batch 39 of 96: Hypervolume = 119.02, time = 2.97s.
Batch 40 of 96: Hypervolume = 119.02, time = 2.77s.
Batch 41 of 96: Hypervolume = 119.02, time = 2.39s.
Batch 42 of 96: Hypervolume = 119.02, time = 3.15s.
Batch 43 of 96: Hypervolume = 119.02, time = 3.90s.
Batch 44 of 96: Hypervolume = 119.02, time = 3.08s.
Batch 45 of 96: Hypervolume = 119.02, time = 3.02s.
Batch 46 of 96: Hypervolume = 119.02, time = 5.19s.
Batch 47 of 96: Hypervolume = 119.02, time = 3.00s.
Batch 48 of 96: Hypervolume = 119.02, time = 3.77s.
Batch 49 of 96: Hypervolume = 119.02, time = 3.83s.
Batch 50 of 96: Hypervolume = 119.02, time = 3.36s.
Batch 51 of 96: Hypervolume = 119.02, time = 3.30s.
Batch 52 of 

Batch 15 of 96: Hypervolume = 119.91, time = 2.03s.
Batch 16 of 96: Hypervolume = 119.91, time = 4.54s.
Batch 17 of 96: Hypervolume = 119.91, time = 3.81s.
Batch 18 of 96: Hypervolume = 119.91, time = 2.38s.
Batch 19 of 96: Hypervolume = 119.91, time = 4.92s.
Batch 20 of 96: Hypervolume = 119.91, time = 2.89s.
Batch 21 of 96: Hypervolume = 119.91, time = 3.73s.
Batch 22 of 96: Hypervolume = 120.04, time = 3.32s.
Batch 23 of 96: Hypervolume = 120.04, time = 3.21s.
Batch 24 of 96: Hypervolume = 120.04, time = 3.46s.
Batch 25 of 96: Hypervolume = 120.04, time = 4.22s.
Batch 26 of 96: Hypervolume = 120.04, time = 4.72s.
Batch 27 of 96: Hypervolume = 120.04, time = 4.29s.
Batch 28 of 96: Hypervolume = 120.04, time = 3.46s.
Batch 29 of 96: Hypervolume = 120.04, time = 2.06s.
Batch 30 of 96: Hypervolume = 120.04, time = 4.78s.
Batch 31 of 96: Hypervolume = 120.04, time = 3.04s.
Batch 32 of 96: Hypervolume = 120.04, time = 5.52s.
Batch 33 of 96: Hypervolume = 120.04, time = 3.32s.
Batch 34 of 

Optimizing with NSGA-III
Time taken in total: 0.69s.
Optimizing with NSGA-III
Time taken in total: 0.82s.
Optimizing with NSGA-III
Time taken in total: 0.69s.

Trial 10 of 10 for problem <class '__main__.Problem_ZDT1'> with d = 8
Optimizing with qNEHVI
Batch  1 of 96: Hypervolume = 94.09, time = 1.63s.
Batch  2 of 96: Hypervolume = 95.21, time = 1.35s.
Batch  3 of 96: Hypervolume = 97.22, time = 1.70s.
Batch  4 of 96: Hypervolume = 102.37, time = 1.63s.
Batch  5 of 96: Hypervolume = 105.66, time = 1.62s.
Batch  6 of 96: Hypervolume = 110.40, time = 1.98s.
Batch  7 of 96: Hypervolume = 113.62, time = 2.69s.
Batch  8 of 96: Hypervolume = 117.56, time = 1.94s.
Batch  9 of 96: Hypervolume = 118.77, time = 1.50s.
Batch 10 of 96: Hypervolume = 119.04, time = 2.28s.
Batch 11 of 96: Hypervolume = 119.28, time = 2.44s.
Batch 12 of 96: Hypervolume = 119.28, time = 2.73s.
Batch 13 of 96: Hypervolume = 119.28, time = 2.23s.
Batch 14 of 96: Hypervolume = 119.28, time = 1.82s.
Batch 15 of 96: Hyperv

Batch  8 of 24: Hypervolume = 118.70, time = 7.44s.
Batch  9 of 24: Hypervolume = 118.70, time = 6.53s.
Batch 10 of 24: Hypervolume = 118.70, time = 8.41s.
Batch 11 of 24: Hypervolume = 118.70, time = 8.45s.
Batch 12 of 24: Hypervolume = 118.70, time = 7.98s.
Batch 13 of 24: Hypervolume = 118.70, time = 9.17s.
Batch 14 of 24: Hypervolume = 118.70, time = 8.70s.
Batch 15 of 24: Hypervolume = 118.70, time = 8.25s.
Batch 16 of 24: Hypervolume = 118.70, time = 6.68s.
Batch 17 of 24: Hypervolume = 118.70, time = 8.86s.
Batch 18 of 24: Hypervolume = 118.70, time = 9.42s.
Batch 19 of 24: Hypervolume = 118.70, time = 9.99s.
Batch 20 of 24: Hypervolume = 118.70, time = 9.86s.
Batch 21 of 24: Hypervolume = 118.70, time = 7.43s.
Batch 22 of 24: Hypervolume = 118.70, time = 9.40s.
Batch 23 of 24: Hypervolume = 118.70, time = 9.67s.
Batch 24 of 24: Hypervolume = 118.70, time = 4.54s.
Time taken in total: 181.81s.
Optimizing with NSGA-III
Time taken in total: 0.76s.
Optimizing with NSGA-III
Time tak

In [4]:
ref_point = torch.tensor([11,11], **tkwargs)

N_TRIALS = 10
verbose = True
noise = 0.00

hvs_qnehvi_all0 = []
hvs_qnehvi_all1 = []
hvs_qnehvi_all2 = []

hvs_nsga3_all0 = []
hvs_nsga3_all1 = []
hvs_nsga3_all2 = []
hvs_nsga3_all3 = []

train_qnehvi_all = []
train_nsga3_all = []

problem = Problem_ZDT2

# main loop for each trial/run, random_state will be trial number
for trial in range(1, 10 + 1):
    print(f"\nTrial {trial:>2} of {N_TRIALS} for problem {problem} with d = {dimensions}\n", end="")

    # initialize with a 2*(d+1) sample set

    initial_x = draw_sobol_samples(bounds=problem.bounds,n=1, q=2*(problem.n_var+1), seed=trial).squeeze(0)

    ###############    
    
    hvs_qnehvi, train_qnehvi = optimize_qnehvi_noconstr(problem, ref_point, initial_x,
                                                        N_BATCH=96, BATCH_SIZE=2,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all0.append(hvs_qnehvi)
        
    ###############    

    hvs_qnehvi, train_qnehvi = optimize_qnehvi_noconstr(problem, ref_point, initial_x,
                                                        N_BATCH=48, BATCH_SIZE=4,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all1.append(hvs_qnehvi)

    ###############    

    hvs_qnehvi, train_qnehvi = optimize_qnehvi_noconstr(problem, ref_point, initial_x,
                                                        N_BATCH=24, BATCH_SIZE=8,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all2.append(hvs_qnehvi)
    train_qnehvi_all.append(train_qnehvi)
   
    ###############    

    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=96, pop_size=2, ref_num=2,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all0.append(hvs_nsga3) 
    
    ###############

    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=48, pop_size=4, ref_num=4,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all1.append(hvs_nsga3) 
    
    ###############
    
    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=24, pop_size=8, ref_num=8,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all2.append(hvs_nsga3)
    train_nsga3_all.append(train_nsga3)
    
    ###############
    
    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=12, pop_size=16, ref_num=16,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all3.append(hvs_nsga3) 
    
    ###############

savetxt("ZDT2_hvs_qnehvi_96by2.csv", hvs_qnehvi_all0, delimiter=',')
savetxt("ZDT2_hvs_qnehvi_48by4.csv", hvs_qnehvi_all1, delimiter=',')
savetxt("ZDT2_hvs_qnehvi_24by8.csv", hvs_qnehvi_all2, delimiter=',')    

savetxt("ZDT2_hvs_nsga3_96by2.csv", hvs_nsga3_all0, delimiter=',')
savetxt("ZDT2_hvs_nsga3_48by4.csv", hvs_nsga3_all1, delimiter=',')
savetxt("ZDT2_hvs_nsga3_24by8.csv", hvs_nsga3_all2, delimiter=',')
savetxt("ZDT2_hvs_nsga3_12by16.csv", hvs_nsga3_all3, delimiter=',')

savetxt("ZDT2_train_qnehvi_24by8.csv", np.array(train_qnehvi_all).reshape(-1), delimiter=',')
savetxt("ZDT2_train_nsga3_24by8.csv", np.array(train_nsga3_all).reshape(-1), delimiter=',')
    
print("ALL DONE!")


Trial  1 of 10 for problem <class '__main__.Problem_ZDT2'> with d = 8
Optimizing with qNEHVI


C:\Users\user\anaconda3\lib\site-packages\botorch\fit.py:143: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is C:\Users\user\AppData\Local\Temp/torch/kernels. This warning will appear only once per process. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\jit_utils.cpp:860.)
  warnings.warn(w.message, w.category)
C:\Users\user\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:1672.)
  Linv = torch.triangular_solve(Eye, 

Batch  1 of 96: Hypervolume = 75.88, time = 4.10s.
Batch  2 of 96: Hypervolume = 76.26, time = 2.80s.
Batch  3 of 96: Hypervolume = 79.72, time = 1.88s.
Batch  4 of 96: Hypervolume = 83.80, time = 1.90s.
Batch  5 of 96: Hypervolume = 87.74, time = 2.49s.
Batch  6 of 96: Hypervolume = 94.71, time = 2.19s.
Batch  7 of 96: Hypervolume = 101.23, time = 2.14s.
Batch  8 of 96: Hypervolume = 101.38, time = 2.07s.
Batch  9 of 96: Hypervolume = 107.95, time = 4.09s.
Batch 10 of 96: Hypervolume = 113.68, time = 2.43s.
Batch 11 of 96: Hypervolume = 113.79, time = 2.00s.
Batch 12 of 96: Hypervolume = 114.78, time = 3.41s.
Batch 13 of 96: Hypervolume = 115.16, time = 1.88s.
Batch 14 of 96: Hypervolume = 115.19, time = 1.80s.
Batch 15 of 96: Hypervolume = 115.19, time = 1.69s.
Batch 16 of 96: Hypervolume = 115.19, time = 1.91s.
Batch 17 of 96: Hypervolume = 115.19, time = 2.41s.
Batch 18 of 96: Hypervolume = 115.19, time = 2.66s.
Batch 19 of 96: Hypervolume = 115.19, time = 3.71s.
Batch 20 of 96: Hy

Batch 13 of 24: Hypervolume = 120.14, time = 6.77s.
Batch 14 of 24: Hypervolume = 120.14, time = 5.98s.
Batch 15 of 24: Hypervolume = 120.14, time = 7.30s.
Batch 16 of 24: Hypervolume = 120.14, time = 7.78s.
Batch 17 of 24: Hypervolume = 120.14, time = 9.26s.
Batch 18 of 24: Hypervolume = 120.14, time = 8.09s.
Batch 19 of 24: Hypervolume = 120.14, time = 8.01s.
Batch 20 of 24: Hypervolume = 120.14, time = 9.36s.
Batch 21 of 24: Hypervolume = 120.14, time = 8.70s.
Batch 22 of 24: Hypervolume = 120.14, time = 8.52s.
Batch 23 of 24: Hypervolume = 120.14, time = 10.98s.
Batch 24 of 24: Hypervolume = 120.14, time = 10.91s.
Time taken in total: 177.96s.
Optimizing with NSGA-III
Time taken in total: 0.74s.
Optimizing with NSGA-III
Time taken in total: 0.68s.
Optimizing with NSGA-III
Time taken in total: 0.72s.
Optimizing with NSGA-III
Time taken in total: 0.74s.

Trial  2 of 10 for problem <class '__main__.Problem_ZDT2'> with d = 8
Optimizing with qNEHVI
Batch  1 of 96: Hypervolume = 79.91, t

Batch 44 of 48: Hypervolume = 119.90, time = 3.66s.
Batch 45 of 48: Hypervolume = 119.90, time = 4.58s.
Batch 46 of 48: Hypervolume = 119.90, time = 5.20s.
Batch 47 of 48: Hypervolume = 119.90, time = 2.21s.
Batch 48 of 48: Hypervolume = 119.90, time = 2.61s.
Time taken in total: 183.64s.
Optimizing with qNEHVI
Batch  1 of 24: Hypervolume = 82.68, time = 5.04s.
Batch  2 of 24: Hypervolume = 93.21, time = 5.85s.
Batch  3 of 24: Hypervolume = 105.57, time = 6.42s.
Batch  4 of 24: Hypervolume = 115.05, time = 5.58s.
Batch  5 of 24: Hypervolume = 115.81, time = 5.70s.
Batch  6 of 24: Hypervolume = 115.81, time = 6.33s.
Batch  7 of 24: Hypervolume = 115.81, time = 6.38s.
Batch  8 of 24: Hypervolume = 115.81, time = 7.50s.
Batch  9 of 24: Hypervolume = 115.81, time = 6.91s.
Batch 10 of 24: Hypervolume = 115.81, time = 7.65s.
Batch 11 of 24: Hypervolume = 115.81, time = 7.13s.
Batch 12 of 24: Hypervolume = 115.81, time = 7.44s.
Batch 13 of 24: Hypervolume = 115.81, time = 8.57s.
Batch 14 of 2

...

Batch 51 of 96: Hypervolume = 110.68, time = 2.68s.
Batch 52 of 96: Hypervolume = 110.68, time = 3.22s.
Batch 53 of 96: Hypervolume = 110.68, time = 2.67s.
Batch 54 of 96: Hypervolume = 110.68, time = 3.24s.
Batch 55 of 96: Hypervolume = 110.68, time = 4.74s.
Batch 56 of 96: Hypervolume = 110.68, time = 5.74s.
Batch 57 of 96: Hypervolume = 110.68, time = 3.08s.
Batch 58 of 96: Hypervolume = 110.68, time = 3.14s.
Batch 59 of 96: Hypervolume = 110.68, time = 3.52s.
Batch 60 of 96: Hypervolume = 110.68, time = 2.31s.
Batch 61 of 96: Hypervolume = 110.68, time = 3.01s.
Batch 62 of 96: Hypervolume = 110.68, time = 4.40s.
Batch 63 of 96: Hypervolume = 110.68, time = 2.07s.
Batch 64 of 96: Hypervolume = 110.68, time = 1.88s.
Batch 65 of 96: Hypervolume = 110.68, time = 3.46s.
Batch 66 of 96: Hypervolume = 110.68, time = 2.22s.
Batch 67 of 96: Hypervolume = 110.68, time = 2.65s.
Batch 68 of 96: Hypervolume = 110.68, time = 2.16s.
Batch 69 of 96: Hypervolume = 110.68, time = 2.66s.
Batch 70 of 

Batch 33 of 96: Hypervolume = 110.76, time = 3.34s.
Batch 34 of 96: Hypervolume = 110.76, time = 3.66s.
Batch 35 of 96: Hypervolume = 110.76, time = 4.22s.
Batch 36 of 96: Hypervolume = 110.76, time = 3.47s.
Batch 37 of 96: Hypervolume = 110.76, time = 2.99s.
Batch 38 of 96: Hypervolume = 110.76, time = 3.82s.
Batch 39 of 96: Hypervolume = 110.76, time = 3.17s.
Batch 40 of 96: Hypervolume = 110.76, time = 2.66s.
Batch 41 of 96: Hypervolume = 110.76, time = 6.58s.
Batch 42 of 96: Hypervolume = 110.76, time = 3.07s.
Batch 43 of 96: Hypervolume = 110.76, time = 3.95s.
Batch 44 of 96: Hypervolume = 110.76, time = 4.24s.
Batch 45 of 96: Hypervolume = 110.76, time = 2.95s.
Batch 46 of 96: Hypervolume = 110.76, time = 4.06s.
Batch 47 of 96: Hypervolume = 110.76, time = 4.84s.
Batch 48 of 96: Hypervolume = 110.76, time = 4.48s.
Batch 49 of 96: Hypervolume = 110.76, time = 3.41s.
Batch 50 of 96: Hypervolume = 110.76, time = 3.76s.
Batch 51 of 96: Hypervolume = 110.76, time = 2.26s.
Batch 52 of 

Batch 15 of 96: Hypervolume = 120.04, time = 2.34s.
Batch 16 of 96: Hypervolume = 120.04, time = 2.17s.
Batch 17 of 96: Hypervolume = 120.04, time = 3.88s.
Batch 18 of 96: Hypervolume = 120.04, time = 3.70s.
Batch 19 of 96: Hypervolume = 120.04, time = 3.22s.
Batch 20 of 96: Hypervolume = 120.04, time = 2.44s.
Batch 21 of 96: Hypervolume = 120.04, time = 4.46s.
Batch 22 of 96: Hypervolume = 120.04, time = 3.30s.
Batch 23 of 96: Hypervolume = 120.04, time = 3.82s.
Batch 24 of 96: Hypervolume = 120.04, time = 3.08s.
Batch 25 of 96: Hypervolume = 120.04, time = 3.73s.
Batch 26 of 96: Hypervolume = 120.04, time = 2.35s.
Batch 27 of 96: Hypervolume = 120.04, time = 2.12s.
Batch 28 of 96: Hypervolume = 120.04, time = 3.07s.
Batch 29 of 96: Hypervolume = 120.04, time = 2.35s.
Batch 30 of 96: Hypervolume = 120.04, time = 3.55s.
Batch 31 of 96: Hypervolume = 120.04, time = 2.47s.
Batch 32 of 96: Hypervolume = 120.04, time = 2.63s.
Batch 33 of 96: Hypervolume = 120.04, time = 2.83s.
Batch 34 of 

Time taken in total: 0.68s.
Optimizing with NSGA-III
Time taken in total: 0.82s.
Optimizing with NSGA-III
Time taken in total: 0.73s.

Trial 10 of 10 for problem <class '__main__.Problem_ZDT2'> with d = 8
Optimizing with qNEHVI
Batch  1 of 96: Hypervolume = 77.68, time = 1.22s.
Batch  2 of 96: Hypervolume = 80.08, time = 1.57s.
Batch  3 of 96: Hypervolume = 83.56, time = 1.62s.
Batch  4 of 96: Hypervolume = 88.19, time = 1.75s.
Batch  5 of 96: Hypervolume = 94.08, time = 2.65s.
Batch  6 of 96: Hypervolume = 101.44, time = 2.53s.
Batch  7 of 96: Hypervolume = 108.40, time = 1.54s.
Batch  8 of 96: Hypervolume = 115.03, time = 2.79s.
Batch  9 of 96: Hypervolume = 115.99, time = 1.63s.
Batch 10 of 96: Hypervolume = 115.99, time = 4.05s.
Batch 11 of 96: Hypervolume = 115.99, time = 1.88s.
Batch 12 of 96: Hypervolume = 115.99, time = 1.52s.
Batch 13 of 96: Hypervolume = 115.99, time = 2.28s.
Batch 14 of 96: Hypervolume = 115.99, time = 1.55s.
Batch 15 of 96: Hypervolume = 116.08, time = 3.29

Batch  9 of 24: Hypervolume = 116.66, time = 6.65s.
Batch 10 of 24: Hypervolume = 116.78, time = 8.35s.
Batch 11 of 24: Hypervolume = 116.78, time = 7.63s.
Batch 12 of 24: Hypervolume = 116.78, time = 4.24s.
Batch 13 of 24: Hypervolume = 116.78, time = 8.04s.
Batch 14 of 24: Hypervolume = 116.78, time = 7.70s.
Batch 15 of 24: Hypervolume = 116.78, time = 9.20s.
Batch 16 of 24: Hypervolume = 116.78, time = 9.03s.
Batch 17 of 24: Hypervolume = 116.78, time = 7.88s.
Batch 18 of 24: Hypervolume = 116.78, time = 7.66s.
Batch 19 of 24: Hypervolume = 116.78, time = 8.58s.
Batch 20 of 24: Hypervolume = 116.78, time = 9.80s.
Batch 21 of 24: Hypervolume = 116.78, time = 9.98s.
Batch 22 of 24: Hypervolume = 116.78, time = 9.47s.
Batch 23 of 24: Hypervolume = 116.78, time = 8.52s.
Batch 24 of 24: Hypervolume = 116.78, time = 8.23s.
Time taken in total: 177.71s.
Optimizing with NSGA-III
Time taken in total: 0.71s.
Optimizing with NSGA-III
Time taken in total: 0.67s.
Optimizing with NSGA-III
Time ta

In [4]:
ref_point = torch.tensor([11,11], **tkwargs)

N_TRIALS = 10
verbose = True
noise = 0.00

hvs_qnehvi_all0 = []
hvs_qnehvi_all1 = []
hvs_qnehvi_all2 = []

hvs_nsga3_all0 = []
hvs_nsga3_all1 = []
hvs_nsga3_all2 = []
hvs_nsga3_all3 = []

train_qnehvi_all = []
train_nsga3_all = []

problem = Problem_ZDT3

# main loop for each trial/run, random_state will be trial number
for trial in range(1, 10 + 1):
    print(f"\nTrial {trial:>2} of {N_TRIALS} for problem {problem} with d = {dimensions}\n", end="")

    # initialize with a 2*(d+1) sample set

    initial_x = draw_sobol_samples(bounds=problem.bounds,n=1, q=2*(problem.n_var+1), seed=trial).squeeze(0)

    ###############    
    
    hvs_qnehvi, train_qnehvi = optimize_qnehvi_noconstr(problem, ref_point, initial_x,
                                                        N_BATCH=96, BATCH_SIZE=2,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all0.append(hvs_qnehvi)
        
    ###############    

    hvs_qnehvi, train_qnehvi = optimize_qnehvi_noconstr(problem, ref_point, initial_x,
                                                        N_BATCH=48, BATCH_SIZE=4,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all1.append(hvs_qnehvi)

    ###############    

    hvs_qnehvi, train_qnehvi = optimize_qnehvi_noconstr(problem, ref_point, initial_x,
                                                        N_BATCH=24, BATCH_SIZE=8,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all2.append(hvs_qnehvi)
    train_qnehvi_all.append(train_qnehvi)
    
    ###############    

    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=96, pop_size=2, ref_num=2,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all0.append(hvs_nsga3) 
    
    ###############

    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=48, pop_size=4, ref_num=4,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all1.append(hvs_nsga3) 
    
    ###############
    
    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=24, pop_size=8, ref_num=8,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all2.append(hvs_nsga3)
    train_nsga3_all.append(train_nsga3)
    
    ###############
    
    hvs_nsga3, train_nsga3 = optimize_nsga3_noconstr(problem, ref_point, initial_x,
                                                     N_BATCH=12, pop_size=16, ref_num=16,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all3.append(hvs_nsga3) 
    
    ###############

savetxt("ZDT3_hvs_qnehvi_96by2.csv", hvs_qnehvi_all0, delimiter=',')
savetxt("ZDT3_hvs_qnehvi_48by4.csv", hvs_qnehvi_all1, delimiter=',')
savetxt("ZDT3_hvs_qnehvi_24by8.csv", hvs_qnehvi_all2, delimiter=',')    

savetxt("ZDT3_hvs_nsga3_96by2.csv", hvs_nsga3_all0, delimiter=',')
savetxt("ZDT3_hvs_nsga3_48by4.csv", hvs_nsga3_all1, delimiter=',')
savetxt("ZDT3_hvs_nsga3_24by8.csv", hvs_nsga3_all2, delimiter=',')
savetxt("ZDT3_hvs_nsga3_12by16.csv", hvs_nsga3_all3, delimiter=',')

savetxt("ZDT3_train_qnehvi_24by8.csv", np.array(train_qnehvi_all).reshape(-1), delimiter=',')
savetxt("ZDT3_train_nsga3_24by8.csv", np.array(train_nsga3_all).reshape(-1), delimiter=',')
    
print("ALL DONE!")


Trial  1 of 10 for problem <class '__main__.Problem_ZDT3'> with d = 8
Optimizing with qNEHVI


C:\Users\user\anaconda3\lib\site-packages\botorch\fit.py:143: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is C:\Users\user\AppData\Local\Temp/torch/kernels. This warning will appear only once per process. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\jit_utils.cpp:860.)
  warnings.warn(w.message, w.category)
C:\Users\user\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:1672.)
  Linv = torch.triangular_solve(Eye, 

Batch  1 of 96: Hypervolume = 114.03, time = 3.84s.
Batch  2 of 96: Hypervolume = 114.03, time = 2.31s.
Batch  3 of 96: Hypervolume = 114.03, time = 1.92s.
Batch  4 of 96: Hypervolume = 114.97, time = 2.00s.
Batch  5 of 96: Hypervolume = 115.34, time = 2.40s.
Batch  6 of 96: Hypervolume = 115.37, time = 2.35s.
Batch  7 of 96: Hypervolume = 115.38, time = 3.49s.
Batch  8 of 96: Hypervolume = 115.38, time = 1.48s.
Batch  9 of 96: Hypervolume = 115.38, time = 3.31s.
Batch 10 of 96: Hypervolume = 115.38, time = 4.14s.
Batch 11 of 96: Hypervolume = 115.38, time = 3.80s.
Batch 12 of 96: Hypervolume = 115.38, time = 4.32s.
Batch 13 of 96: Hypervolume = 115.38, time = 5.15s.
Batch 14 of 96: Hypervolume = 115.38, time = 3.20s.
Batch 15 of 96: Hypervolume = 115.38, time = 3.38s.
Batch 16 of 96: Hypervolume = 115.38, time = 2.91s.
Batch 17 of 96: Hypervolume = 115.39, time = 4.21s.
Batch 18 of 96: Hypervolume = 115.39, time = 4.94s.
Batch 19 of 96: Hypervolume = 115.39, time = 4.80s.
Batch 20 of 

Batch 13 of 24: Hypervolume = 120.53, time = 11.23s.
Batch 14 of 24: Hypervolume = 120.60, time = 10.68s.
Batch 15 of 24: Hypervolume = 120.70, time = 11.91s.
Batch 16 of 24: Hypervolume = 120.70, time = 10.92s.
Batch 17 of 24: Hypervolume = 120.71, time = 11.51s.
Batch 18 of 24: Hypervolume = 120.71, time = 12.62s.
Batch 19 of 24: Hypervolume = 120.71, time = 13.02s.
Batch 20 of 24: Hypervolume = 120.71, time = 12.27s.
Batch 21 of 24: Hypervolume = 120.73, time = 9.57s.
Batch 22 of 24: Hypervolume = 120.73, time = 13.29s.
Batch 23 of 24: Hypervolume = 120.74, time = 9.62s.
Batch 24 of 24: Hypervolume = 120.75, time = 13.61s.
Time taken in total: 249.86s.
Optimizing with NSGA-III
Time taken in total: 0.75s.
Optimizing with NSGA-III
Time taken in total: 0.71s.
Optimizing with NSGA-III
Time taken in total: 0.76s.
Optimizing with NSGA-III
Time taken in total: 0.73s.

Trial  2 of 10 for problem <class '__main__.Problem_ZDT3'> with d = 8
Optimizing with qNEHVI
Batch  1 of 96: Hypervolume = 

Batch 43 of 48: Hypervolume = 126.34, time = 4.77s.
Batch 44 of 48: Hypervolume = 126.34, time = 2.70s.
Batch 45 of 48: Hypervolume = 126.34, time = 5.98s.
Batch 46 of 48: Hypervolume = 126.34, time = 4.53s.
Batch 47 of 48: Hypervolume = 126.36, time = 5.93s.
Batch 48 of 48: Hypervolume = 126.36, time = 5.30s.
Time taken in total: 199.98s.
Optimizing with qNEHVI
Batch  1 of 24: Hypervolume = 116.13, time = 4.57s.
Batch  2 of 24: Hypervolume = 116.60, time = 7.30s.
Batch  3 of 24: Hypervolume = 116.60, time = 9.13s.
Batch  4 of 24: Hypervolume = 116.85, time = 10.60s.
Batch  5 of 24: Hypervolume = 117.04, time = 11.15s.
Batch  6 of 24: Hypervolume = 117.75, time = 10.50s.
Batch  7 of 24: Hypervolume = 119.37, time = 11.75s.
Batch  8 of 24: Hypervolume = 119.39, time = 10.32s.
Batch  9 of 24: Hypervolume = 122.49, time = 11.01s.
Batch 10 of 24: Hypervolume = 122.52, time = 10.56s.
Batch 11 of 24: Hypervolume = 122.52, time = 13.37s.
Batch 12 of 24: Hypervolume = 124.14, time = 12.63s.
Ba

...

Batch 45 of 96: Hypervolume = 128.44, time = 3.56s.
Batch 46 of 96: Hypervolume = 128.44, time = 3.93s.
Batch 47 of 96: Hypervolume = 128.44, time = 3.17s.
Batch 48 of 96: Hypervolume = 128.44, time = 3.43s.
Batch 49 of 96: Hypervolume = 128.44, time = 3.68s.
Batch 50 of 96: Hypervolume = 128.44, time = 5.29s.
Batch 51 of 96: Hypervolume = 128.45, time = 2.14s.
Batch 52 of 96: Hypervolume = 128.45, time = 2.88s.
Batch 53 of 96: Hypervolume = 128.45, time = 3.50s.
Batch 54 of 96: Hypervolume = 128.45, time = 3.30s.
Batch 55 of 96: Hypervolume = 128.45, time = 3.16s.
Batch 56 of 96: Hypervolume = 128.45, time = 2.70s.
Batch 57 of 96: Hypervolume = 128.45, time = 2.75s.
Batch 58 of 96: Hypervolume = 128.45, time = 3.05s.
Batch 59 of 96: Hypervolume = 128.45, time = 3.05s.
Batch 60 of 96: Hypervolume = 128.45, time = 2.32s.
Batch 61 of 96: Hypervolume = 128.45, time = 3.12s.
Batch 62 of 96: Hypervolume = 128.46, time = 2.44s.
Batch 63 of 96: Hypervolume = 128.46, time = 3.37s.
Batch 64 of 

Batch 26 of 96: Hypervolume = 117.71, time = 3.29s.
Batch 27 of 96: Hypervolume = 117.71, time = 3.78s.
Batch 28 of 96: Hypervolume = 117.71, time = 3.96s.
Batch 29 of 96: Hypervolume = 117.71, time = 4.14s.
Batch 30 of 96: Hypervolume = 118.55, time = 4.65s.
Batch 31 of 96: Hypervolume = 118.55, time = 3.90s.
Batch 32 of 96: Hypervolume = 118.55, time = 5.57s.
Batch 33 of 96: Hypervolume = 118.55, time = 3.63s.
Batch 34 of 96: Hypervolume = 118.58, time = 6.09s.
Batch 35 of 96: Hypervolume = 118.59, time = 4.66s.
Batch 36 of 96: Hypervolume = 118.59, time = 5.73s.
Batch 37 of 96: Hypervolume = 118.59, time = 2.91s.
Batch 38 of 96: Hypervolume = 118.59, time = 3.82s.
Batch 39 of 96: Hypervolume = 120.25, time = 3.30s.
Batch 40 of 96: Hypervolume = 120.25, time = 4.40s.
Batch 41 of 96: Hypervolume = 120.25, time = 5.82s.
Batch 42 of 96: Hypervolume = 120.25, time = 3.43s.
Batch 43 of 96: Hypervolume = 120.25, time = 3.93s.
Batch 44 of 96: Hypervolume = 120.25, time = 4.22s.
Batch 45 of 

Batch  7 of 96: Hypervolume = 115.87, time = 2.48s.
Batch  8 of 96: Hypervolume = 115.87, time = 3.15s.
Batch  9 of 96: Hypervolume = 116.04, time = 4.93s.
Batch 10 of 96: Hypervolume = 116.04, time = 4.92s.
Batch 11 of 96: Hypervolume = 116.04, time = 2.82s.
Batch 12 of 96: Hypervolume = 116.04, time = 4.20s.
Batch 13 of 96: Hypervolume = 116.13, time = 4.05s.
Batch 14 of 96: Hypervolume = 116.52, time = 4.82s.
Batch 15 of 96: Hypervolume = 116.52, time = 4.22s.
Batch 16 of 96: Hypervolume = 118.19, time = 4.97s.
Batch 17 of 96: Hypervolume = 118.19, time = 4.55s.
Batch 18 of 96: Hypervolume = 119.26, time = 4.84s.
Batch 19 of 96: Hypervolume = 119.27, time = 4.45s.
Batch 20 of 96: Hypervolume = 120.33, time = 5.15s.
Batch 21 of 96: Hypervolume = 120.33, time = 4.26s.
Batch 22 of 96: Hypervolume = 121.35, time = 4.34s.
Batch 23 of 96: Hypervolume = 121.35, time = 3.62s.
Batch 24 of 96: Hypervolume = 121.36, time = 3.59s.
Batch 25 of 96: Hypervolume = 121.38, time = 3.40s.
Batch 26 of 

Batch 19 of 24: Hypervolume = 126.44, time = 10.89s.
Batch 20 of 24: Hypervolume = 126.44, time = 13.33s.
Batch 21 of 24: Hypervolume = 126.45, time = 10.77s.
Batch 22 of 24: Hypervolume = 126.45, time = 7.06s.
Batch 23 of 24: Hypervolume = 126.47, time = 12.08s.
Batch 24 of 24: Hypervolume = 126.48, time = 13.50s.
Time taken in total: 275.76s.
Optimizing with NSGA-III
Time taken in total: 0.79s.
Optimizing with NSGA-III
Time taken in total: 0.74s.
Optimizing with NSGA-III
Time taken in total: 0.96s.
Optimizing with NSGA-III
Time taken in total: 0.83s.

Trial 10 of 10 for problem <class '__main__.Problem_ZDT3'> with d = 8
Optimizing with qNEHVI
Batch  1 of 96: Hypervolume = 116.21, time = 1.66s.
Batch  2 of 96: Hypervolume = 116.21, time = 2.10s.
Batch  3 of 96: Hypervolume = 116.28, time = 1.68s.
Batch  4 of 96: Hypervolume = 116.49, time = 2.04s.
Batch  5 of 96: Hypervolume = 116.49, time = 1.80s.
Batch  6 of 96: Hypervolume = 116.49, time = 2.01s.
Batch  7 of 96: Hypervolume = 116.4

Batch  1 of 24: Hypervolume = 116.21, time = 6.30s.
Batch  2 of 24: Hypervolume = 116.70, time = 8.20s.
Batch  3 of 24: Hypervolume = 117.10, time = 9.89s.
Batch  4 of 24: Hypervolume = 117.15, time = 10.11s.
Batch  5 of 24: Hypervolume = 118.41, time = 10.76s.
Batch  6 of 24: Hypervolume = 118.41, time = 11.02s.
Batch  7 of 24: Hypervolume = 120.60, time = 12.18s.
Batch  8 of 24: Hypervolume = 123.64, time = 12.61s.
Batch  9 of 24: Hypervolume = 123.64, time = 11.45s.
Batch 10 of 24: Hypervolume = 127.87, time = 11.81s.
Batch 11 of 24: Hypervolume = 127.92, time = 11.75s.
Batch 12 of 24: Hypervolume = 127.92, time = 11.43s.
Batch 13 of 24: Hypervolume = 127.92, time = 10.44s.
Batch 14 of 24: Hypervolume = 127.92, time = 12.30s.
Batch 15 of 24: Hypervolume = 127.96, time = 12.97s.
Batch 16 of 24: Hypervolume = 127.96, time = 13.35s.
Batch 17 of 24: Hypervolume = 127.97, time = 12.56s.
Batch 18 of 24: Hypervolume = 127.98, time = 13.06s.
Batch 19 of 24: Hypervolume = 128.00, time = 13.7

In [4]:
ref_point = torch.tensor([11,11], **tkwargs)

N_TRIALS = 10
verbose = True
noise = 0.00

hvs_qnehvi_all0 = []
hvs_qnehvi_all1 = []
hvs_qnehvi_all2 = []

hvs_nsga3_all0 = []
hvs_nsga3_all1 = []
hvs_nsga3_all2 = []
hvs_nsga3_all3 = []

train_qnehvi_all = []
train_nsga3_all = []

problem = Problem_MW7

# main loop for each trial/run, random_state will be trial number
for trial in range(1, 10 + 1):
    print(f"\nTrial {trial:>2} of {N_TRIALS} for problem {problem} with d = {dimensions}\n", end="")

    # initialize with a 2*(d+1) sample set

    initial_x = draw_sobol_samples(bounds=problem.bounds,n=1, q=2*(problem.n_var+1), seed=trial).squeeze(0)

    ###############    
    
    hvs_qnehvi, train_qnehvi = optimize_qnehvi(problem, ref_point, initial_x,
                                                        N_BATCH=96, BATCH_SIZE=2,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all0.append(hvs_qnehvi)
        
    ###############    

    hvs_qnehvi, train_qnehvi = optimize_qnehvi(problem, ref_point, initial_x,
                                                        N_BATCH=48, BATCH_SIZE=4,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all1.append(hvs_qnehvi)

    ###############    

    hvs_qnehvi, train_qnehvi = optimize_qnehvi(problem, ref_point, initial_x,
                                                        N_BATCH=24, BATCH_SIZE=8,
                                                        random_state=trial, noise=noise, verbose=verbose)
    hvs_qnehvi_all2.append(hvs_qnehvi)
    train_qnehvi_all.append(train_qnehvi)
    
    ###############    

    hvs_nsga3, train_nsga3 = optimize_nsga3(problem, ref_point, initial_x,
                                                     N_BATCH=96, pop_size=2, ref_num=2,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all0.append(hvs_nsga3) 
    
    ###############

    hvs_nsga3, train_nsga3 = optimize_nsga3(problem, ref_point, initial_x,
                                                     N_BATCH=48, pop_size=4, ref_num=4,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all1.append(hvs_nsga3) 
    
    ###############
    
    hvs_nsga3, train_nsga3 = optimize_nsga3(problem, ref_point, initial_x,
                                                     N_BATCH=24, pop_size=8, ref_num=8,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all2.append(hvs_nsga3)
    train_nsga3_all.append(train_nsga3)
    
    ###############
    
    hvs_nsga3, train_nsga3 = optimize_nsga3(problem, ref_point, initial_x,
                                                     N_BATCH=12, pop_size=16, ref_num=16,
                                                     random_state=trial, noise=noise, verbose=False)
    hvs_nsga3_all3.append(hvs_nsga3) 
    
    ###############

savetxt("MW7_hvs_qnehvi_96by2.csv", hvs_qnehvi_all0, delimiter=',')
savetxt("MW7_hvs_qnehvi_48by4.csv", hvs_qnehvi_all1, delimiter=',')
savetxt("MW7_hvs_qnehvi_24by8.csv", hvs_qnehvi_all2, delimiter=',')    

savetxt("MW7_hvs_nsga3_96by2.csv", hvs_nsga3_all0, delimiter=',')
savetxt("MW7_hvs_nsga3_48by4.csv", hvs_nsga3_all1, delimiter=',')
savetxt("MW7_hvs_nsga3_24by8.csv", hvs_nsga3_all2, delimiter=',')
savetxt("MW7_hvs_nsga3_12by16.csv", hvs_nsga3_all3, delimiter=',')

savetxt("MW7_train_qnehvi_24by8.csv", np.array(train_qnehvi_all).reshape(-1), delimiter=',')
savetxt("MW7_train_nsga3_24by8.csv", np.array(train_nsga3_all).reshape(-1), delimiter=',')
    
print("ALL DONE!")


Trial  1 of 10 for problem <class '__main__.Problem_MW7'> with d = 8
Optimizing with qNEHVI


C:\Users\user\anaconda3\lib\site-packages\botorch\fit.py:143: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is C:\Users\user\AppData\Local\Temp/torch/kernels. This warning will appear only once per process. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\jit_utils.cpp:860.)
  warnings.warn(w.message, w.category)
C:\Users\user\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:1672.)
  Linv = torch.triangular_solve(Eye, 

Batch  1 of 96: Hypervolume = 0.00, time = 12.10s.
Batch  2 of 96: Hypervolume = 0.00, time = 15.52s.
Batch  3 of 96: Hypervolume = 0.00, time = 13.40s.


KeyboardInterrupt: 